<a href="https://colab.research.google.com/github/vktiwary007/stock-move-eureka/blob/main/stockMovement0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.4.1
Eager mode:  True
Hub version:  0.11.0
GPU is available


In [3]:
import io
df = pd.read_csv('Apple240.csv', skiprows=0)

In [4]:
df[:10]

,Source,News,Movement
0,seekingalpha.com,Bearish Calls Lumentum (NASDAQ: LITE ): It's a...,1
1,seekingalpha.com,The areas to be concerned are that there are f...,1
2,seekingalpha.com,Amazon will soon resume selling the Apple TV (...,1
3,seekingalpha.com,"Facing questions from tech analysts, Apple (NA...",0
4,www.thestreet.com,Cramer and the AAP team say investors need to ...,0
5,seekingalpha.com,One day after Apple acknowledged that it has b...,1
6,www.thestreet.com,1 . -- U.S. stock futures turned lower on Tues...,1
7,seekingalpha.com,"Meanwhile, eight more lawsuits have been filed...",1
8,seekingalpha.com,Shares in Apple and several of its suppliers f...,1
9,www.thestreet.com,1 . -- U.S. stock futures traded slightly high...,0


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['News'].values, df.iloc[:, -1].values, test_size = 0.2, random_state = 0)

train_examples, train_labels = np.asarray(X_train), np.asarray(y_train)
test_examples, test_labels = np.asarray(X_test), np.asarray(y_test)

In [6]:
train_examples[:10]

array(['Q1 ’16: -4.1%, -6.1% Source: Refinitiv IBES consensus estimates as of 10/22/18 Summary/conclusion: S&P 500 earnings growth is expected to slow to 10% in calendar 2019, and the numbers have been telling investors that for quite some time, as this blog discussed here . I don’t think this is the reason behind this market downdraft that started on October 1 ’18, but we’ll see. China is a growing issue – their growth has slowed and that has to come home to roost in US earnings at some point if it hasn’t already. Companies give some idea of next year’s guidance starting with the 3rd quarter earnings reports every year, but the real or hard guidance occurs with the January-February conference calls that contain 4th quarter results. If we average the above 2019 results, the Tech sector – over the 4 quarters – is expected to average 8.2% EPS growth and 5% revenue growth, inline with the S&P 500’s expected average of 9% EPS growth and 5% revenue growth. As we lap the corporate income tax

In [7]:
train_labels[:10]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1])

In [8]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))

Training entries: 29251, test entries: 7313


In [9]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 1.2355471 ,  0.11719096, -0.19826788,  0.62281877, -0.49590316,
        -0.738948  ,  0.7471023 ,  0.61302024, -0.501993  ,  0.2771705 ,
         0.07472976,  0.25415528,  0.35520852, -0.36702234,  0.62300056,
        -0.7748881 , -0.56931186,  0.45307308,  0.5203001 , -0.03229528,
        -0.09448511, -0.35797906,  0.9466501 , -0.08701267,  0.12805209,
         0.23130086, -0.6522287 ,  0.44974676, -0.20601137, -0.37608024,
         0.21680853, -0.1402651 ,  0.773747  , -0.58659416, -0.07478948,
        -0.08432465,  0.37597057, -0.5652265 ,  0.03625645, -0.664764  ,
        -0.5988652 ,  0.03650454,  0.3608431 ,  0.5561065 , -0.47489214,
        -0.78675216, -0.35815898, -0.03418034, -0.09439875,  0.2157843 ],
       [ 0.20898803, -0.37149653,  0.1520238 ,  0.07758274,  0.06931368,
        -0.05553383, -0.02514357,  0.19448043, -0.05831357, -0.20240895,
        -0.28871888,  0.04763449,  0.11879189, -0.27784148,  0.5745588 ,
 

In [10]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [12]:
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [13]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40
38/38 [==============================] - 5s 50ms/step - loss: 0.6930 - accuracy: 0.5278 - val_loss: 0.6904 - val_accuracy: 0.5293
Epoch 2/40
38/38 [==============================] - 2s 43ms/step - loss: 0.6794 - accuracy: 0.5720 - val_loss: 0.6892 - val_accuracy: 0.5343
Epoch 3/40
38/38 [==============================] - 2s 43ms/step - loss: 0.6659 - accuracy: 0.6026 - val_loss: 0.6883 - val_accuracy: 0.5342
Epoch 4/40
38/38 [==============================] - 2s 42ms/step - loss: 0.6451 - accuracy: 0.6451 - val_loss: 0.6868 - val_accuracy: 0.5453
Epoch 5/40
38/38 [==============================] - 2s 43ms/step - loss: 0.6137 - accuracy: 0.6957 - val_loss: 0.6905 - val_accuracy: 0.5438
Epoch 6/40
38/38 [==============================] - 2s 43ms/step - loss: 0.5723 - accuracy: 0.7240 - val_loss: 0.6947 - val_accuracy: 0.5552
Epoch 7/40
38/38 [==============================] - 2s 43ms/step - loss: 0.5294 - accuracy: 0.7555 - val_loss: 0.7079 - val_accuracy: 0.5668
Epoch 8/40
38

In [15]:
results = model.evaluate(test_examples, test_labels)

print(results)

229/229 [==============================] - 1s 3ms/step - loss: 1.6692 - accuracy: 0.5869
[1.669185757637024, 0.5869000554084778]
